# Stacking the CLAMATO dataset

In [2]:
# imports
from astropy import constants as const

from linetools.spectra import utils as ltsu

from specdb.specdb import IgmSpec
from xastropy.xutils import xdebug as xdb

## Test spec DB

In [3]:
clamato = IgmSpec(db_file=os.getenv('DROPBOX_DIR')+'/cl2016_redux/clamato.hdf5')

Database is CLAMATO
Created on 2017-Jan-23


In [4]:
clamato.hdf['LRIS/spec'].shape

(237,)

## Grab a set of the spectra

In [5]:
ids = clamato.qcat.cat['CLAM_ID']
ids[0:3]

0
1
2


### Lya coverage

In [7]:
gds = clamato.cat['zem'] > 1.7
np.sum(gds)

231

In [8]:
rows = clamato['LRIS'].ids_to_firstrow(ids[gds])
len(rows)

231

In [10]:
spec, meta = clamato['LRIS'].grab_specmeta(rows)

Staged 231 spectra totalling 0.0155232 Gb
Loaded spectra


/Users/xavier/local/Python/linetools/linetools/spectra/xspectrum1d.py:248: UserWarning: No unit given to wavelength, assuming Angstroms.
  warnings.warn("No unit given to wavelength, assuming Angstroms.")


In [11]:
spec

<XSpectrum1D: file=none, nspec=231, select=0, wvmin=3850.23 Angstrom, wvmax=5603.31 Angstrom>

In [12]:
meta[0:5]

RA_GROUP,DEC_GROUP,STYPE,zem_GROUP,sig_zem,flag_zem,DATE-OBS,R,DISPERSER,EPOCH,GROUP_ID,CLAM_ID,NPIX,WV_MIN,WV_MAX,SPEC_FILE,INSTR,TELESCOPE
float64,float64,str3,float64,float64,str8,str10,float64,str8,float64,int64,int64,int64,float64,float64,str71,str5,str6
150.2357,2.1661,QSO,2.128,0.0,CLAMATO,2016-02-12,1000.0,600/4000,2000.0,0,0,1375,3850.2,5603.3,/u/xavier/Dropbox//cl2016_redux/spec///./c16_11_zph2.3_12434_F.fits,LRISb,Keck-I
150.2045,2.1710,QSO,2.370,0.0,CLAMATO,2016-02-12,1000.0,600/4000,2000.0,1,1,1816,3346.8,5603.1,/u/xavier/Dropbox//cl2016_redux/spec///./c16_11_zph2.4_12707_F.fits,LRISb,Keck-I
150.1827,2.1548,QSO,2.436,0.0,CLAMATO,2016-02-12,1000.0,600/4000,2000.0,2,2,1989,3103.5,5520.5,/u/xavier/Dropbox//cl2016_redux/spec///./c16_11_zph2.4_12732_F.fits,LRISb,Keck-I
150.1698,2.1409,QSO,2.170,0.0,CLAMATO,2016-02-12,1000.0,600/4000,2000.0,3,3,1835,3102.7,5308.6,/u/xavier/Dropbox//cl2016_redux/spec///./c16_11_zph2.4_12751_F.fits,LRISb,Keck-I
150.2277,2.2360,QSO,2.493,0.0,CLAMATO,2016-02-12,1000.0,600/4000,2000.0,4,4,1516,3682.6,5603.5,/u/xavier/Dropbox//cl2016_redux/spec///./c16_11_zph2.5_12304_F.fits,LRISb,Keck-I


In [14]:
np.min(meta['zem_GROUP'])

1.74

## Rebin

In [15]:
zarr = meta['zem_GROUP']

In [17]:
reload(ltsu)
rest_spec = ltsu.rebin_to_rest(spec, zarr, 200*u.km/u.s, debug=False, grow_bad_sig=True)

In [18]:
rest_spec.select=2
rest_spec.plot()

In [19]:
rest_spec.sig
# That last pixel is a bit worrisome

<Quantity [ 0., 0., 0.,...,  0., 0., 0.]>

## Stack

In [20]:
reload(ltsu)
stack = ltsu.smash_spectra(rest_spec, debug=False)

In [ ]:
stack.plot(xspec=True)

/Users/xavier/local/Python/linetools/linetools/spectra/xspectrum1d.py:347: UserWarning: This spectrum does not contain an input error array
  warnings.warn("This spectrum does not contain an input error array")


button=1, x=650.000000, y=352.000000, xdata=1493.550004, ydata=0.10523


## Quick figure

## Tests

In [32]:
nspec, npix = 3, 100
wave = np.outer(np.ones(nspec), np.arange(npix))
flux = np.ones_like(wave)
sig = np.ones_like(wave)

In [33]:
from linetools.spectra.xspectrum1d import XSpectrum1D

In [34]:
new_spec = XSpectrum1D(wave, flux, sig=sig)